In [2]:
!pip install tensorflow

In [3]:
import urllib.request

url = "https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip"
urllib.request.urlretrieve(url, "10_food_classes_10_percent.zip")


('10_food_classes_10_percent.zip', <http.client.HTTPMessage at 0x79505a43c6b0>)

In [4]:
import zipfile

with zipfile.ZipFile('10_food_classes_10_percent.zip', 'r') as zip_ref:
  zip_ref.extractall()

In [5]:
# Configuration Path File for train and test data

train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"

In [6]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Training Set
training_set = image_dataset_from_directory(
    directory = train_dir,
    image_size = (224,224),
    label_mode = 'categorical',
    batch_size = 32,
    validation_split = 0.3, # we take 70% data for training
    subset = 'training',
    seed = 42
    )

# Validation_test
validation_test = image_dataset_from_directory(
    directory = train_dir,
    image_size = (224, 224),
    label_mode = 'categorical',
    batch_size = 32,
    validation_split = 0.3, # and another 30% for validation
    subset = 'validation',
    seed = 42
)

# Test_set
test_set = image_dataset_from_directory(
    directory = test_dir,
    image_size = (224,224),
    label_mode = 'categorical'
)

Found 750 files belonging to 10 classes.
Using 525 files for training.
Found 750 files belonging to 10 classes.
Using 225 files for validation.
Found 2500 files belonging to 10 classes.


In [7]:
from tensorflow.keras.applications import EfficientNetV2B0

# We use pre-trained model
pretrained_model = EfficientNetV2B0(
    weights = 'imagenet', include_top = False,
    pooling = 'avg'
)

# Lock all the network parameters in the pre-trained model so that they cannot be changed or trained
pretrained_model.trainable = False

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [8]:
from tensorflow.keras.layers import Input
inputs = Input(shape = (224,224,3), name = 'input_layer')

# Connected the input layer to the pre-trained model
revised_model_3 = pretrained_model(inputs)

from tensorflow.keras.layers import Dense

# Connected the output layer to the pre-trained model
outputs = Dense( units= 10, activation = 'softmax', # for this case, we use 10 neuron
                name = 'output_layer')(revised_model_3)

In [9]:
# Linked the input and output layers to create the model
from tensorflow.keras import Model
model_3 = Model(inputs, outputs, name = 'model_3')

# Compile the model with Loss Function and optimizer
model_3.compile(loss = 'categorical_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])

# Checked the model
model_3.summary()

Model: "model_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-b0 (Functional)  │ (None, 1280)           │     5,919,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 10)             │        12,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,932,122 (22.63 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 5,919,312 (22.58 MB)

In [10]:
# Create two type of callbacks
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

es_callback = EarlyStopping(monitor = 'val_loss', patience = 5)
mpt_callback = ModelCheckpoint(filepath = 'best_model_3.keras',
save_best_only = True)


In [11]:
# Train Model
model_3.fit(training_set, epochs = 100,
            validation_data = validation_test,
            callbacks = [es_callback, mpt_callback])

Epoch 1/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 69s 3s/step - accuracy: 0.2523 - loss: 2.1767 - val_accuracy: 0.5911 - val_loss: 1.6542
Epoch 2/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 43s 3s/step - accuracy: 0.6991 - loss: 1.4302 - val_accuracy: 0.7333 - val_loss: 1.2170
Epoch 3/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 90s 3s/step - accuracy: 0.8163 - loss: 1.0034 - val_accuracy: 0.7822 - val_loss: 1.0024
Epoch 4/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 51s 3s/step - accuracy: 0.8320 - loss: 0.8188 - val_accuracy: 0.7867 - val_loss: 0.8943
Epoch 5/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 73s 3s/step - accuracy: 0.8809 - loss: 0.6672 - val_accuracy: 0.7911 - val_loss: 0.8152
Epoch 6/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 51s 3s/step - accuracy: 0.8684 - loss: 0.6051 - val_accuracy: 0.8133 - val_loss: 0.7643
Epoch 7/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.8902 - loss: 0.5552 - val_accuracy: 0.8044 - val_loss: 0.7290
Epoch 8/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 75s 3s/step - accuracy: 0.8865 - loss: 0.4975 - val_accuracy: 0.8044 - v

In [14]:
from tensorflow.keras.models import load_model

# Load best model that was saved by ModelCheckPoint
best_model_3 = load_model('best_model_3.keras')

# Testing model
best_model_3.evaluate(test_set)

79/79 ━━━━━━━━━━━━━━━━━━━━ 146s 2s/step - accuracy: 0.8731 - loss: 0.4138


[0.40236133337020874, 0.8740000128746033]

With a much smaller Training Set (750 images vs. 7,500 originally) and keeping the convolution layer parameters of EfficientNetV2B0 unchanged, we still achieved very high accuracy.

